# OpenAIRE Graph University Overview

This notebook fetches summary statistics per university using the [OpenAIRE Graph API](https://graph.openaire.eu/docs/apis/graph-api/). For every listed university we compare three perspectives:

- **A. Publications affiliated to the university** – filter on the OpenAIRE OpenORG identifier.
- **B. Publications collected by the main (CRIS) data source** – filter on the CRS/data source identifier.
- **C. Publications collected by the secondary repository** – filter on the repository identifier when available.

For each perspective we retrieve counts for funding/projects, data sources, and research products split into publications, datasets, software, and other research outputs. The notebook separates the data collection steps so that each can be re-run independently when debugging or iterating.


In [ ]:
# install required packages (if not already installed)

!pip install pandas matplotlib openpyxl python-dotenv requests tqdm


## 1. Imports and reused constants


In [12]:
from concurrent.futures import ThreadPoolExecutor
import os
import csv
import time
from copy import deepcopy
from io import StringIO
from pathlib import Path
from typing import Any, Dict, Optional, Union

import matplotlib.pyplot as plt
import pandas as pd
import requests
from tqdm.auto import tqdm
from dotenv import load_dotenv

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda value: f"{value:,.0f}")

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

if not CLIENT_ID or not CLIENT_SECRET:
    raise RuntimeError(
        "Missing OpenAIRE credentials. Set CLIENT_ID and CLIENT_SECRET in the environment."
    )

BASE_URL = "https://api.openaire.eu/graph"
TOKEN_URL = "https://aai.openaire.eu/oidc/token"
API_USER_AGENT = "OpenAIRE-tools overview-stats notebook"
API_PAUSE_SECONDS = 0.1  # throttle requests a bit to stay within rate limits
TOKEN_REFRESH_BUFFER = 60  # refresh the token one minute before expiration

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

PRODUCT_TYPE_LABELS = {
    "publication": "Publications",
    "dataset": "Research data",
    "software": "Research software",
    "other": "Other research products",
}

METRIC_ORDER = [
    "Funding / Projects",
    "Data sources",
    *PRODUCT_TYPE_LABELS.values(),
]

COMPARISON_LONG_PATH = DATA_DIR / "comparison_long.csv"
COMPARISON_PIVOT_PATH = DATA_DIR / "comparison_pivot.csv"

_access_token: Optional[str] = None
_access_token_expiry: float = 0.0

print("Setup complete.")


Setup complete.


## 2. Parse the university reference table
The raw table below mirrors the values supplied in the request. We reshape it into a structured list so that the rest of the notebook can iterate over the entries.


In [ ]:
# 2. Load the NL organizations reference table
# Download the latest NL organizations baseline from the Google Sheets URL

# URL of the published Google Sheets NL organizations baseline
nl_orgs_baseline_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTDQiWDIaI1SZkPTMNCovicBhA-nQND1drXoUKvrG1O_Ga3hLDRvmQZao_TvNgmNQ/pub?output=xlsx"
# Local path to store the downloaded baseline
NL_ORGS_BASELINE_PATH = DATA_DIR / "nl_orgs_baseline.xlsx"
# Name of the sheet containing the NL organizations data
NL_ORGS_SHEET_NAME = "nl-orgs"

# Function to download the NL organizations baseline
def download_nl_orgs_baseline(url: str, dest: Path) -> Path:
    """Download the latest NL organizations baseline and store it locally."""
    # Ensure the destination directory exists
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print(f"Using existing baseline file: {dest}")
        return dest
        
    print(f"Downloading baseline file from {url}")
    # Download the file
    response = requests.get(url, timeout=120)
    response.raise_for_status()
    dest.write_bytes(response.content)
    print(f"Saved baseline file to: {dest}")
    return dest

# Download the NL organizations baseline
download_nl_orgs_baseline(nl_orgs_baseline_url, NL_ORGS_BASELINE_PATH)

# Set the table path to the downloaded baseline
TABLE_PATH = NL_ORGS_BASELINE_PATH

# Load and parse the NL organizations reference table
def load_university_table(path: Union[str, Path], sheet_name: str) -> pd.DataFrame:
    """Load the NL organizations reference table and normalise empty values to blanks."""
    table_path = Path(path)
    if not table_path.exists():
        raise FileNotFoundError(f"Reference table not found: {table_path}")
    excel_suffixes = {".xlsx", ".xls"}
    if table_path.suffix.lower() in excel_suffixes:
        # Load the Excel file
        df = pd.read_excel(table_path, sheet_name=sheet_name, dtype=str, keep_default_na=False)
    else:
        # Load as a TSV/CSV file
        df = pd.read_csv(table_path, sep="	", dtype=str, keep_default_na=False)
    return df.fillna("")

# Helper function to pick the first non-empty value from a list of columns
def _pick(row: pd.Series, *columns: str) -> str:
    for column in columns:
        if column not in row:
            continue
        value = row[column]
        if isinstance(value, str):
            value = value.strip()
        if value:
            return value
    return ""

# Normalize ROR link
def normalise_ror_link(value: str) -> str:
    # Normalize ROR identifier to a full URL
    value = (value or "").strip()
    # If the value is empty after stripping, return an empty string
    if not value:
        return ""
    # If the value already starts with "http", return it as is
    if value.startswith("http"):
        return value
    # If the value starts with "ror.org", prepend "https://"
    if value.startswith("ror.org"):
        return f"https://{value}"
    # Otherwise, assume it's a ROR ID and construct the full URL
    value = value.strip().strip("/")
    # If the value is empty after stripping, return an empty string
    if not value:
        return ""
    # Return the full ROR URL
    return f"https://ror.org/{value}"

# Extract ROR ID from a value
def extract_ror_id(value: str) -> str:
    value = (value or "").strip()
    if not value:
        return ""
    if value.startswith("http"):
        value = value.rstrip("/").split("/")[-1]
    return value

# Parse university rows from the DataFrame
def parse_university_rows(df: pd.DataFrame) -> list[dict[str, Optional[str]]]:
    # Parse university rows from the DataFrame and extract relevant fields
    parsed: list[dict[str, Optional[str]]] = []
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Determine the university name using multiple possible columns
        name = _pick(row, "full_name_in_English", "University", "organization_name")
        # Skip rows without a valid name
        if not name:
            continue
        
        # Extract and normalize ROR information
        ror_id = extract_ror_id(_pick(row, "ROR"))
        # Normalize ROR link
        ror_link = normalise_ror_link(_pick(row, "ROR_LINK") or ror_id)
        # If ROR link is available but ROR ID is not, extract ROR ID from the link
        if ror_link and not ror_id:
            # Extract ROR ID from the normalized ROR link
            ror_id = extract_ror_id(ror_link)

        # Construct the university entry
        entry = {
            "name": name,
            "acronym_EN": _pick(row, "acronym_EN") or None,
            "acronym_AGG": _pick(row, "acronym_AGG") or None,
            "grouping": _pick(row, "main_grouping") or None,
            "ROR": ror_id or None,
            "ROR_LINK": ror_link or None,
            "OpenAIRE_ORG_ID": _pick(row, "OpenAIRE_ORG_ID", "OpenAIRE OpenORG ID", "OpenAIRE OpenORG ID LINK", "openorg_id") or None,
            "main_datasource_id": _pick(row, "OpenAIRE Data Source ID (Main/CRIS)", "OpenAIRE Data Source ID (Main/CRIS) LINK", "main_datasource_id") or None,
            "secondary_datasource_id": _pick(row, "OpenAIRE Data Source (Secondary/Repository)", "OpenAIRE Data Source (Secondary/Repository) LINK", "secondary_datasource_id") or None,
        }
        # Special handling for OpenDOAR IDs
        if entry["main_datasource_id"] and entry["main_datasource_id"].startswith("opendoar"):
            if not entry["secondary_datasource_id"]:
                entry["secondary_datasource_id"] = entry["main_datasource_id"]
                entry["main_datasource_id"] = None
        # Append the parsed entry to the list
        parsed.append(entry)

    # Return the parsed university entries
    return parsed

# Load the NL organizations reference table and parse universities
nl_orgs_df = load_university_table(TABLE_PATH, NL_ORGS_SHEET_NAME)
# Parse universities from the loaded DataFrame
universities = parse_university_rows(nl_orgs_df)
# Display the number of parsed universities
print(f"Parsed {len(universities)} universities from the reference table (loaded from {TABLE_PATH}).")
# Convert the list of universities to a DataFrame for display
universities_df = pd.DataFrame(universities)
# Display the DataFrame of universities
universities_df


Using existing baseline file: data/nl_orgs_baseline.xlsx
Parsed 93 universities from the reference table (loaded from data/nl_orgs_baseline.xlsx).


,name,acronym_EN,acronym_AGG,grouping,ROR,ROR_LINK,OpenAIRE_ORG_ID,main_datasource_id,secondary_datasource_id
0,Data Archiving Networked Services (DANS),DANS,DANS,KNAW,008pnp284,https://ror.org/008pnp284,None,None,None
1,Hubrecht Institute for Developmental Biology a...,HUBRECHT,HUBRECHT,KNAW,023qc4a07,https://ror.org/023qc4a07,None,None,None
2,International Institute of Social History (IISH),IISH,IISH,KNAW,05dq4pp56,https://ror.org/05dq4pp56,None,None,None
3,Royal Netherlands Institute of Southeast Asian...,KITLV,KITLV,KNAW,01bdv4312,https://ror.org/01bdv4312,None,None,None
4,Royal Netherlands Academy of Arts and Sciences,KNAW,KNAW,KNAW,043c0p156,https://ror.org/043c0p156,None,None,None
5,Netherlands Institute for Neuroscience,NIN,NIN,KNAW,05csn2x06,https://ror.org/05csn2x06,None,None,None
6,Netherlands Institute of Ecology (NIOO),NIOO,NIOO,KNAW,01g25jp36,https://ror.org/01g25jp36,None,None,None
7,Westerdijk Fungal Biodiversity Institute,WESTERDIJK,WESTERDIJK,KNAW,030a5r161,https://ror.org/030a5r161,None,None,None
8,Amsterdam UMC,AUMC,AUMC,NFU,05grdyy37,https://ror.org/05grdyy37,None,None,None
9,ErasmusMC,ErasmusMC,EUR/ErasmusMC,NFU,018906e22,https://ror.org/018906e22,None,None,None


## 3. Helper functions for the Graph API
These functions wrap the REST requests and centralise filter construction per scenario. Each call prints nothing by default so we can reuse them freely in later cells.


In [14]:
# Define the different scenarios for metrics collection.
# Each scenario includes a key, label, ID field, and description.
# For each scenario, we will build appropriate filters to query the OpenAIRE Graph API.
# For example, we can collect metrics based on organization affiliation,
# main data source, or secondary repository.
# Each scenario will have its own set of filters.

SCENARIO_DEFS = [
    {
        "key": "organization",
        "label": "A. OpenORG affiliation",
        "id_field": "OpenAIRE_ORG_ID",
        "description": "Publications affiliated to the university (OpenAIRE OpenORG ID)",
    },
    {
        "key": "main_datasource",
        "label": "B. Main/CRIS data source",
        "id_field": "main_datasource_id",
        "description": "Publications collected from the main CRIS data source",
    },
    {
        "key": "secondary_datasource",
        "label": "C. Secondary repository",
        "id_field": "secondary_datasource_id",
        "description": "Publications collected from the secondary / repository data source",
    },
]

# Define filter builders for each scenario.
# Each builder function takes an entity ID and returns a dictionary of filters
# for projects, data sources, and research products.
#   - The filters are used to query the OpenAIRE Graph API for relevant metrics.

FILTER_BUILDERS = {
    "organization": lambda entity_id: {
        "projects": {"relOrganizationId": entity_id},
        "dataSources": {"relOrganizationId": entity_id},
        "researchProducts": {"relOrganizationId": entity_id},
    },
    "main_datasource": lambda entity_id: {
        "projects": {"relCollectedFromDatasourceId": entity_id},
        "dataSources": {"id": entity_id},
        "researchProducts": {"relCollectedFromDatasourceId": entity_id},
    },
    "secondary_datasource": lambda entity_id: {
        "projects": {"relCollectedFromDatasourceId": entity_id},
        "dataSources": {"id": entity_id},
        "researchProducts": {"relCollectedFromDatasourceId": entity_id},
    },
}

# Define an empty metrics dictionary for collecting results.
EMPTY_METRICS = {metric: None for metric in METRIC_ORDER}

# Obtain a cached OpenAIRE access token, refreshing it when needed.
def obtain_access_token() -> str:
    """Return a cached OpenAIRE access token, refreshing it when needed."""
    global _access_token, _access_token_expiry
    now = time.time()
    if _access_token and now < _access_token_expiry:
        return _access_token

    response = requests.post(
        TOKEN_URL,
        data={"grant_type": "client_credentials"},
        auth=(CLIENT_ID, CLIENT_SECRET),
        headers={"User-Agent": API_USER_AGENT},
        timeout=60,
    )
    response.raise_for_status()
    payload = response.json()
    token = payload.get("access_token")
    if not token:
        raise RuntimeError("OpenAIRE token response did not include an access_token.")
    expires_in = int(payload.get("expires_in", 3600))
    _access_token = token
    _access_token_expiry = now + max(expires_in - TOKEN_REFRESH_BUFFER, 0)
    return _access_token

# Invoke the OpenAIRE Graph API and return the decoded JSON payload.
def call_graph_api(path: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """Invoke the OpenAIRE Graph API and return the decoded JSON payload."""
    url = f"{BASE_URL}{path}"
    effective_params = dict(params or {})
    effective_params.setdefault("page", 1)
    effective_params.setdefault("pageSize", 1)

    headers = {
        "User-Agent": API_USER_AGENT,
        "Authorization": f"Bearer {obtain_access_token()}",
    }

    response = requests.get(
        url,
        params=effective_params,
        headers=headers,
        timeout=60,
    )
    response.raise_for_status()
    time.sleep(API_PAUSE_SECONDS)
    return response.json()

# Retrieve the total number of matching records for the supplied endpoint.
def fetch_num_found(path: str, params: Dict[str, Any]) -> Optional[int]:
    """Return the total number of matching records for the supplied endpoint."""
    payload = call_graph_api(path, params)
    header = payload.get("header", {})
    num_found = header.get("numFound")
    return int(num_found) if num_found is not None else None

# Build filters for the given scenario and entity ID.
def build_filters(scenario_key: str, entity_id: str) -> Dict[str, Dict[str, Any]]:
    builder = FILTER_BUILDERS[scenario_key]
    return {name: dict(filters) for name, filters in builder(entity_id).items()}


# Collect metrics for a given scenario and entity ID.
def collect_metrics(scenario_key: str, entity_id: Optional[str]) -> Dict[str, Optional[int]]:
    if not entity_id:
        return deepcopy(EMPTY_METRICS)

    filters = build_filters(scenario_key, entity_id)
    results: Dict[str, Optional[int]] = {}

    results["Funding / Projects"] = fetch_num_found("/v1/projects", filters["projects"])
    results["Data sources"] = fetch_num_found("/v1/dataSources", filters["dataSources"])

    for rp_type, label in PRODUCT_TYPE_LABELS.items():
        rp_params = dict(filters["researchProducts"], type=rp_type)
        results[label] = fetch_num_found("/v2/researchProducts", rp_params)

    return results


In [15]:
from functools import lru_cache

# Cache OpenAIRE ID lookups to avoid redundant API calls

@lru_cache(maxsize=None)
def _fetch_openorg_id(ror_url: str) -> Optional[str]:
    # Query the OpenAIRE Graph API for the organization with the given ROR URL
    try:
        payload = call_graph_api("/v1/organizations", {"pid": ror_url})
    except requests.RequestException as exc:
        print(f"Failed to fetch OpenAIRE ID for {ror_url}: {exc}")
        return None

    # Extract the OpenAIRE organization ID from the response
    results = payload.get("results") or []
    for item in results:
        openorg_id = item.get("id")
        if openorg_id:
            return openorg_id
    return None

# Look up the OpenAIRE organization identifier using a ROR ID or URL.

def fetch_openorg_id_for_ror(ror_value: Optional[str]) -> Optional[str]:
    """Look up the OpenAIRE organization identifier using a ROR ID or URL."""
    if not ror_value:
        return None
    # Normalize the ROR value to a URL
    ror_url = normalise_ror_link(ror_value)
    if not ror_url:
        return None
    # Look up the OpenAIRE organization ID using the normalized ROR URL
    return _fetch_openorg_id(ror_url)


## 4. Test for one university

Run a quick test for the first university in the list to fetch the 
  OpenORG affiliation,
  and the Number of: 
  Funding / Projects,
  Data sources, 
  Products: Total,
  Products: Publications, 
  Products: Research data, 
  Products: Research software,
  Products: Other research products

In [16]:
# Example: collect and display metrics for the first university in the list
test_university = universities[0]
print(f"Testing metrics for: {test_university['name']}")

# Determine the OpenAIRE organization identifier
identifier = test_university.get("OpenAIRE_ORG_ID")
if not identifier:
    # Try to resolve it via the ROR ID/link
    resolved = fetch_openorg_id_for_ror(test_university.get("ROR_LINK") or test_university.get("ROR"))
    identifier = resolved

# Collect and display metrics    
print(f"\nOpenAIRE ID: {identifier}")
print("\nMetrics:")
# Collect metrics for the organization
metrics = collect_metrics("organization", identifier)

# Calculate total research products
research_products_total = sum(
    metrics.get(product_type, 0) or 0 
    for product_type in ["Publications", "Research data", "Research software", "Other research products"]
)
print("Total Research Products:", research_products_total)
print("\nBreakdown by type:")

# Display the collected metrics
for metric, count in metrics.items():
    print(f"{metric}: {count}")

Testing metrics for: Data Archiving Networked Services (DANS)

OpenAIRE ID: openorgs____::a8767e80afb51d63802fdf1ddabeacb0

Metrics:
Total Research Products: 10231

Breakdown by type:
Funding / Projects: 0
Data sources: 16
Publications: 438
Research data: 9561
Research software: 99
Other research products: 133


## 5. Collect metrics for all universities
Loop through every organisation, ensure an OpenAIRE identifier is available, and add this information to new columns. and write to file nl_orgs_openaire.xlsx

In [ ]:
# Enrich each organization with OpenAIRE IDs and metrics
# This may involve multiple API calls and data processing steps.

# Copy the university table so we can enrich it without mutating the original
enriched_df = universities_df.copy()

# Ensure all metric columns exist in the dataframe
metric_columns = [
    "Data sources",
    "Total Research Products",
    "Publications",
    "Research data",
    "Research software",
    "Other research products",
]
# Add missing metric columns with NA values
for column in metric_columns:
    if column not in enriched_df.columns:
        enriched_df[column] = pd.NA


# define paths for output and checkpoint files
output_path = DATA_DIR / "nl_orgs_openaire.xlsx"
checkpoint_path = DATA_DIR / "nl_orgs_openaire.tmp.xlsx"

# define save interval and enriched count
save_every = 5  # write a checkpoint every N organisations
enriched_count = 0 # count of organizations with added OpenAIRE IDs

# define progress iterator
# unit is "org" for organizations
progress = tqdm(enriched_df.index, desc="Enriching OpenAIRE IDs & metrics", unit="org")

# Iterate over each organization to enrich with OpenAIRE IDs and metrics
for processed, idx in enumerate(progress, start=1):
    row = enriched_df.loc[idx]
    identifier = row.get("OpenAIRE_ORG_ID")

    # If no OpenAIRE ID is present, try to resolve it via the ROR ID/link
    if not identifier:
        identifier = fetch_openorg_id_for_ror(row.get("ROR_LINK") or row.get("ROR"))
        if identifier:
            enriched_df.at[idx, "OpenAIRE_ORG_ID"] = identifier
            enriched_count += 1
    
    # Collect metrics for the organization
    metrics = collect_metrics("organization", identifier)

    # Populate the metric columns in the dataframe
    if identifier and metrics:
        for column in metric_columns:
            # Calculate total research products
            if column == "Total Research Products":
                total_products = sum((metrics.get(label) or 0) for label in metric_columns[2:])
                enriched_df.at[idx, column] = total_products
            # Populate other metric columns
            else:
                enriched_df.at[idx, column] = metrics.get(column)
    # If no metrics were found, set all metric columns to NA
    else:
        for column in metric_columns:
            enriched_df.at[idx, column] = pd.NA
    
    # Save a checkpoint at defined intervals
    if processed % save_every == 0:
        enriched_df.to_excel(checkpoint_path, index=False)
        progress.set_postfix(saved_rows=processed)

# Persist the final enriched table
enriched_df.to_excel(output_path, index=False)

# Display summary of the enrichment process
print(f"Added OpenAIRE IDs for {enriched_count} organizations")
# Display the total number of organizations processed
print(f"Saved enriched data to {output_path}")
# Check if a checkpoint file exists
if checkpoint_path.exists():
    print(f"Checkpoint saved to {checkpoint_path}")

# Preview the enriched dataframe
enriched_df.head()

# Display organizations still missing OpenAIRE IDs
missing_ids = enriched_df[enriched_df["OpenAIRE_ORG_ID"].isna() | (enriched_df["OpenAIRE_ORG_ID"] == "")]
if not missing_ids.empty:
    print("Organizations still missing OpenAIRE IDs:")
    for name in missing_ids['name']:
        print(f"- {name}")


In [ ]:
# preview the enriched dataframe
enriched_df.head()

## 5. Collect metrics for all universities
Loop through every organisation, ensure an OpenAIRE identifier is available, and gather the scenario metrics.

In [ ]:
aggregated_results: list[dict[str, Any]] = []
batch_enriched = 0
missing_openorg_ids: list[str] = []

# Ensure every university has an OpenAIRE identifier before fetching metrics
for university in universities:
    if not university.get("OpenAIRE_ORG_ID"):
        enriched = fetch_openorg_id_for_ror(university.get("ROR_LINK") or university.get("ROR"))
        if enriched:
            university["OpenAIRE_ORG_ID"] = enriched
            batch_enriched += 1
        else:
            missing_openorg_ids.append(university["name"])

universities_df = pd.DataFrame(universities)

def collect_university_metrics(university: dict[str, Any]) -> dict[str, Any]:
    uni_entry = {
        "info": university,
        "scenarios": {},
    }
    for scenario in SCENARIO_DEFS:
        identifier = university.get(scenario["id_field"])
        metrics = collect_metrics(scenario["key"], identifier)
        uni_entry["scenarios"][scenario["key"]] = metrics
    return uni_entry

# Run the per-university metric collection in parallel to speed up API calls
with ThreadPoolExecutor(max_workers=6) as executor:
    for uni_entry in tqdm(
        executor.map(collect_university_metrics, universities),
        total=len(universities),
        desc="Collecting metrics",
        unit="org",
    ):
        aggregated_results.append(uni_entry)

for entry in aggregated_results:
    university = entry["info"]
    print(f"
=== {university['name']} ===")
    for scenario in SCENARIO_DEFS:
        identifier = university.get(scenario["id_field"])
        metrics = entry["scenarios"][scenario["key"]]
        label = scenario["label"]
        identifier_display = identifier if identifier else "no identifier provided"
        print(f"{label} ({identifier_display}):")
        for metric in METRIC_ORDER:
            print(f"  {metric}: {metrics.get(metric)}")

print(
    f"
Completed data collection for all universities. "
    f"Enriched OpenAIRE IDs for {batch_enriched} organisations."
)
if missing_openorg_ids:
    print("Missing OpenAIRE IDs for:")
    for name in missing_openorg_ids:
        print(f"  - {name}")


## 6. Assemble and save the comparison table
Reshape the collected metrics into long and pivot formats and cache the CSV files under `data/`.

In [ ]:
records: list[dict[str, Any]] = []
for entry in aggregated_results:
    base = entry["info"]
    ror_value = base.get("ROR")
    if not ror_value and base.get("ROR_LINK"):
        ror_value = base["ROR_LINK"].rstrip("/").split("/")[-1]
    for scenario in SCENARIO_DEFS:
        metrics = entry["scenarios"].get(scenario["key"], {})
        for metric in METRIC_ORDER:
            records.append(
                {
                    "University": base["name"],
                    "ROR": ror_value,
                    "Scenario": scenario["label"],
                    "Metric": metric,
                    "Count": metrics.get(metric),
                }
            )

comparison_df = pd.DataFrame(records)
comparison_pivot = (
    comparison_df.pivot_table(
        index=["University", "ROR"],
        columns=["Scenario", "Metric"],
        values="Count",
    )
    .sort_index(axis=1, level=0)
    .astype("Int64")
)

comparison_df.to_csv(COMPARISON_LONG_PATH, index=False)
comparison_pivot.to_csv(COMPARISON_PIVOT_PATH)
print(f"Saved long-format table to {COMPARISON_LONG_PATH}")
print(f"Saved pivot table to {COMPARISON_PIVOT_PATH}")
comparison_pivot


## 7. Reload cached comparison tables
Reload the CSV outputs so downstream analysis can rely on the cached results without rerunning the API calls.

In [ ]:
comparison_df = pd.read_csv(COMPARISON_LONG_PATH)
if "Count" in comparison_df.columns:
    comparison_df["Count"] = comparison_df["Count"].astype("Int64")

comparison_pivot = pd.read_csv(
    COMPARISON_PIVOT_PATH,
    header=[0, 1],
    index_col=[0, 1],
)
comparison_pivot.columns = pd.MultiIndex.from_tuples(comparison_pivot.columns)
comparison_pivot = comparison_pivot.astype("Int64")
print(f"Reloaded tables from {COMPARISON_LONG_PATH} and {COMPARISON_PIVOT_PATH}")
comparison_pivot


## 8. Visualise publication counts
Plot the publication totals per scenario to highlight differences between affiliation and data-source perspectives.

In [ ]:
publication_df = comparison_df[comparison_df["Metric"] == "Publications"].copy()
wide_publications = (
    publication_df.pivot(index="University", columns="Scenario", values="Count")
    .reindex(comparison_df["University"].unique())
)

fig, ax = plt.subplots(figsize=(12, 6))
wide_publications.plot(kind="bar", ax=ax)
ax.set_xlabel("University")
ax.set_ylabel("Number of publications")
ax.set_title("Publications by affiliation vs. data sources")
ax.legend(title="Scenario")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

print("Note: Missing bars indicate that the source may not be connected to OpenAIRE or does not expose publications.")
